# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fac44870e80>
├── 'a' --> tensor([[-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622]])
└── 'x' --> <FastTreeValue 0x7fac44870eb0>
    └── 'c' --> tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                        [-0.5181, -0.8591,  0.2371, -0.4617],
                        [-0.1982,  0.5489, -0.4995,  0.3107]])

In [4]:
t.a

tensor([[-0.4472, -0.2492, -1.1582],
        [-0.3714, -0.3435, -0.6622]])

In [5]:
%timeit t.a

65.4 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fac44870e80>
├── 'a' --> tensor([[-0.7737,  0.6632, -0.9945],
│                   [-1.1975, -0.9079, -0.6268]])
└── 'x' --> <FastTreeValue 0x7fac44870eb0>
    └── 'c' --> tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                        [-0.5181, -0.8591,  0.2371, -0.4617],
                        [-0.1982,  0.5489, -0.4995,  0.3107]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.0283 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4472, -0.2492, -1.1582],
               [-0.3714, -0.3435, -0.6622]]),
    x: Batch(
           c: tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107]]),
       ),
)

In [10]:
b.a

tensor([[-0.4472, -0.2492, -1.1582],
        [-0.3714, -0.3435, -0.6622]])

In [11]:
%timeit b.a

57.4 ns ± 0.0546 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.6296, -0.7235,  2.9548],
               [ 0.7702,  0.6441,  0.3842]]),
    x: Batch(
           c: tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.0685 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

844 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 46.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 489 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fab97576ee0>
├── 'a' --> tensor([[[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]],
│           
│                   [[-0.4472, -0.2492, -1.1582],
│                    [-0.3714, -0.3435, -0.6622]]])
└── 'x' --> <FastTreeValue 0x7fac448644c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 70.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fac4485f9d0>
├── 'a' --> tensor([[-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622],
│                   [-0.4472, -0.2492, -1.1582],
│                   [-0.3714, -0.3435, -0.6622]])
└── 'x' --> <FastTreeValue 0x7fac4485f340>
    └── 'c' --> tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                        [-0.5181, -0.8591,  0.2371, -0.4617],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 80.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.3993, -0.7059, -1.0204,  1.5734],
                       [-0.5181, -0.8591,  0.2371, -0.4617],
                       [-0.1982,  0.5489, -0.4995,  0.3107]],
              
                      [[-2.3993, -0.7059, -1.0204,  1.5734],
                       [-0.5181, -0.8591,  0.2371, -0.4617],
                       [-0.1982,  0.5489, -0.4995,  0.3107]],
              
                      [[-2.3993, -0.7059, -1.0204,  1.5734],
                       [-0.5181, -0.8591,  0.2371, -0.4617],
                       [-0.1982,  0.5489, -0.4995,  0.3107]],
              
                      [[-2.3993, -0.7059, -1.0204,  1.5734],
                       [-0.5181, -0.8591,  0.2371, -0.4617],
                       [-0.1982,  0.5489, -0.4995,  0.3107]],
              
                      [[-2.3993, -0.7059, -1.0204,  1.5734],
                       [-0.5181, -0.8591,  0.2371, -0.4617],
                       [-0.1982,  0.5489, -0.4995,  0.3107]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107],
                      [-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107],
                      [-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107],
                      [-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107],
                      [-2.3993, -0.7059, -1.0204,  1.5734],
                      [-0.5181, -0.8591,  0.2371, -0.4617],
                      [-0.1982,  0.5489, -0.4995,  0.3107],
                      [-2.3993, -0.7059, -1.0204,  1.5734],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 486 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 482 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
